Load necessary libraries and functions:

In [1]:
import numpy as np
from scipy.sparse import coo_matrix
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from numpy import linalg as LA
from scipy.sparse.linalg import svds, eigs
from sklearn.decomposition import PCA
from scipy.sparse import csr_matrix

def convertNansToZeros(ma):
    nan_elements = np.flatnonzero(np.isnan(ma.data))
    if len(nan_elements) > 0:
        ma.data[nan_elements] = 0
    return ma


def convertInfsToZeros(ma):
    inf_elements = np.flatnonzero(np.isinf(ma.data))
    if len(inf_elements) > 0:
        ma.data[inf_elements] = 0
    return ma

Use the main.sh script located in /home/garner1/Work/pipelines/gpseq-hic to generate the merged HiC and GPseq dataset. 

In [ ]:
'''Here we evaluate the correlation matrix of HiC contacts row\col-normalized using GPseq, without balancing first'''

datamat = np.loadtxt('/home/garner1/Work/dataset/gpseq+hic/gpseq.1M.chr1.bincount/BICRO48_TK77_10min_GG__cutsiteLoc-umiCount.transCorrected.bed.1M.chr1.dat',usecols=(0,1,5))

i = datamat[:,0]
j = datamat[:,1]
data = datamat[:,2]
s = max([int(max(i)), int(max(j))])
counts = coo_matrix((data, (i, j)), shape=(s+1, s+1))
counts = counts.todense()

'''Evaluate the expected minor diagonals values'''
expected = np.zeros(shape=[counts.shape[0]-1,])
for offset in xrange(counts.shape[0]-1):
    expected[offset] = np.trace(counts, offset=offset+1)/(counts.shape[0]-offset) #it is important to divide by the offdiag mean, not only the sum of the offdiag

'''Normalize by minor diagonals sum'''
for row in xrange(counts.shape[0]):
    for col in xrange(row+1,counts.shape[1]):
        counts[row,col] = counts[row,col]/expected[col-row-1]
        counts[col,row] = counts[row,col]
counts = convertNansToZeros(coo_matrix(counts)).todense()
counts = convertInfsToZeros(coo_matrix(counts)).todense()

'''Take the log'''
counts = np.log(counts)    
counts = convertNansToZeros(coo_matrix(counts)).todense()
counts = convertInfsToZeros(coo_matrix(counts)).todense()

'''Take the Pearson correlation matrix'''
corrmatrix = np.corrcoef(counts)
corrmatrix = convertNansToZeros(coo_matrix(corrmatrix)).todense()
corrmatrix = convertInfsToZeros(coo_matrix(corrmatrix)).todense()

evals, eigs = LA.eig(corrmatrix)

# %matplotlib
plt.figure(0)
cmap = sns.diverging_palette(220, 20, sep=20, as_cmap=True)
ax = sns.heatmap(counts, center=counts.mean(),cmap=cmap)
plt.savefig('heatmap_on.png')

plt.figure(1)
fig, ax = plt.subplots()
y = eigs[:,0].getA1()
y[122:143] = np.nan
y = y-np.nanmean(y)
ax.scatter(x=range(len(y)), y=y, c=np.sign(y), cmap=cmap)
plt.savefig('eigvec_on.png')
ax = sns.barplot(x=range(len(eigs[:,0].getA1())),y=eigs[:,0].getA1())
plt.show(ax)

plt.figure(2)
cmap = sns.diverging_palette(220, 20, sep=20, as_cmap=True)
y = eigs[:,0].getA1()
mat = np.outer(y,y)
ax = sns.heatmap(mat, center=mat.mean(),cmap=cmap)
plt.savefig('IeigenSpace_10min.png')


In [ ]:
'''Here we evaluate the correlation matrix of HiC contacts with KR balancing and oe entries'''

datamat = np.loadtxt('/home/garner1/Work/dataset/hic/gpseq.1M.chr1.bincount/BICRO48_TK80_on_GG__cutsiteLoc-umiCount.transCorrected.bed.1M.chr1.dat',usecols=(0,1,2))
i = datamat[:,0]
j = datamat[:,1]
data = datamat[:,2]
s = max([int(max(i)), int(max(j))])
counts = coo_matrix((data, (i, j)), shape=(s+1, s+1))
counts = counts.todense()

'''Take the log'''
counts = np.log(counts)    
counts = convertNansToZeros(coo_matrix(counts)).todense()
counts = convertInfsToZeros(coo_matrix(counts)).todense()

'''Take the Pearson correlation matrix'''
corrmatrix = np.corrcoef(counts)
corrmatrix = convertNansToZeros(coo_matrix(corrmatrix)).todense()
corrmatrix = convertInfsToZeros(coo_matrix(corrmatrix)).todense()

evals, eigs = LA.eig(corrmatrix)

plt.figure(0)
cmap = sns.diverging_palette(220, 20, sep=20, as_cmap=True)
ax = sns.heatmap(corrmatrix, center=corrmatrix.mean(),cmap=cmap)
plt.savefig('heatmap_KR-oe.png')

fig, ax = plt.subplots()
y = -eigs[:,0].getA1()
y[122:143] = np.nan
y = y-np.nanmean(y)
ax.scatter(x=range(len(y)), y=y, c=np.sign(y), cmap=cmap)
plt.savefig('eigvec_KR-oe.png')
# ax = sns.barplot(x=range(len(eigs[:,0].getA1())),y=eigs[:,0].getA1())
# plt.show(ax)

plt.figure(2)
cmap = sns.diverging_palette(220, 20, sep=20, as_cmap=True)
y = eigs[:,0].getA1()
mat = np.outer(y,y)
ax = sns.heatmap(mat, center=mat.mean(),cmap=cmap)
plt.savefig('IeigenSpace_KR-oe.png')


INPUT: combined HiC and GPseq dataset.

From HiC we take the raw data (unbalanced and observed).

GPseq counts are used to evaluate the indipendent probability assumption.

The mutual information matrix is then constructed, keeping the information on the centrality and the time of digestion (by using different time of digestion GPseq data): 
$$ P_{HiC}(i,j)\log \frac{P_{HiC}(i,j)}{P_{GPseq}(i)P_{GPseq}(j)} $$

In [ ]:
import glob

for chromosome in range(1,9)+range(10,22):
    for time in ['10min','15min','30min','on']:
        print chromosome, time
        run = '58'
        experiment = '*'
#         time = duration
        chrom = str(chromosome)
        normalization = 'KR'
        tipo = 'observed'
        resolution = '1M'

        directory = 'gpseq.'+resolution+'.'+normalization+'.'+tipo+'.chr'+chrom+'.bincount'
        filename = 'BICRO'+run+'_TK'+experiment+'_'+time+'_GG__cutsiteLoc-umiCount.transCorrected.bed.'+resolution+'.chr'+chrom+'.dat'

        datamat = np.loadtxt(glob.glob('/home/garner1/Work/dataset/gpseq+hic/'+directory+'/'+filename)[0],usecols=(0,1,2,3,4))
        tags = np.loadtxt('/home/garner1/Work/dataset/gpseq+hic/tagged_with_centrality/chr'+chrom+'.bc'+run)

        tags_1 = tags[tags[:,1] == 1]
        tags_2 = tags[tags[:,1] == 2]
        tags_3 = tags[tags[:,1] == 3]
        tags_4 = tags[tags[:,1] == 4]

        '''Define contact matrix'''
        i = datamat[:,0]
        j = datamat[:,1]
        p_ij = datamat[:,2]
        p_ij = p_ij / p_ij.sum()
        p_i = datamat[:,3]
        p_i = p_i / p_i.sum()
        p_j = datamat[:,4]
        p_j = p_j / p_j.sum()
        s = max([int(max(i)), int(max(j))])
        pairwise = coo_matrix((p_ij, (i, j)), shape=(s+1, s+1)).todense()
        indip = coo_matrix((p_i*p_j, (i, j)), shape=(s+1, s+1)).todense()
        
        info = np.log2(pairwise / indip)
        info = convertNansToZeros(coo_matrix(info)).todense()
        info = convertInfsToZeros(coo_matrix(info)).todense()

        counts = np.array(pairwise) * np.array(info)  #the mutual information matrix

        '''Filter with respect to centrality'''
        counts_1 = np.zeros(counts.shape)
        counts_2 = np.zeros(counts.shape)
        counts_3 = np.zeros(counts.shape)
        counts_4 = np.zeros(counts.shape)
        for row in xrange(counts.shape[0]):
            for col in xrange(counts.shape[1]):
                    if (row in tags_1[:,0]) and (col in tags_1[:,0]):
                        counts_1[row,col] = counts[row,col]
                    else:
                        counts_1[row,col] = 0
                    if (row in tags_2[:,0]) and (col in tags_2[:,0]):
                        counts_2[row,col] = counts[row,col]
                    else:
                        counts_2[row,col] = 0
                    if (row in tags_3[:,0]) and (col in tags_3[:,0]):
                        counts_3[row,col] = counts[row,col]
                    else:
                        counts_3[row,col] = 0
                    if (row in tags_4[:,0]) and (col in tags_4[:,0]):
                        counts_4[row,col] = counts[row,col]
                    else:
                        counts_4[row,col] = 0
        locals()['mi_'+time+'_'+'chr'+chrom] = [np.sum(counts_1),np.sum(counts_2),np.sum(counts_3),np.sum(counts_4)]
    fig, ax = plt.subplots()
    y = locals()['mi_10min_'+'chr'+chrom]
    ax.scatter(x=range(len(y)),y=y,color='blue',label='10min')
    y = locals()['mi_15min_'+'chr'+chrom]
    ax.scatter(x=range(len(y)),y=y,color='green',label='15min')
    y = locals()['mi_30min_'+'chr'+chrom]
    ax.scatter(x=range(len(y)),y=y,color='red',label='30min')
    y = locals()['mi_on_'+'chr'+chrom]
    ax.scatter(x=range(len(y)),y=y,color='orange',label='ON')
    ax.set_xticks([0,1,2,3])
    ax.set_xticklabels(["periphery","II layer","I layer","center"])
    ax.set_title('Mutual information for chr'+chrom)
    plt.legend()
    plt.savefig('MI_chr'+chrom+'.png')
    
# fig, ax = plt.subplots()
# y = np.array([mi_10min,mi_15min,mi_30min,mi_on])[:,3]
# ax.scatter(x=range(4), y=y,color='blue')
# ax.set_xticks([0,1,2,3])
# ax.set_xticklabels(["10min","15min","30min","ON"])
# ax.set_title('Mutual information for chr1 at the center')
# plt.savefig('MI_center_chr1.png')

In the next block we consider HiC raw data and define independent probabilities from the marginal of the joint probability (from HiC), ie there is no GPseq data here.

In [ ]:
import glob

for chromosome in range(1,9)+range(10,22):
    print chromosome, time
    run = '58'
    experiment = '*'
#         time = duration
    chrom = str(chromosome)
    normalization = 'KR'
    tipo = 'observed'
    resolution = '1M'

    directory = 'gpseq.'+resolution+'.'+normalization+'.'+tipo+'.chr'+chrom+'.bincount'
    filename = 'BICRO'+run+'_TK'+experiment+'_'+time+'_GG__cutsiteLoc-umiCount.transCorrected.bed.'+resolution+'.chr'+chrom+'.dat'

    datamat = np.loadtxt(glob.glob('/home/garner1/Work/dataset/gpseq+hic/'+directory+'/'+filename)[0],usecols=(0,1,2,3,4))
    tags = np.loadtxt('/home/garner1/Work/dataset/gpseq+hic/tagged_with_centrality/chr'+chrom+'.bc'+run)

    tags_1 = tags[tags[:,1] == 1]
    tags_2 = tags[tags[:,1] == 2]
    tags_3 = tags[tags[:,1] == 3]
    tags_4 = tags[tags[:,1] == 4]

    '''Define the matrix of joint probabilities'''
    i = datamat[:,0]
    j = datamat[:,1]
    p_ij = datamat[:,2]
    p_ij = p_ij / p_ij.sum()
    s = max([int(max(i)), int(max(j))])
    pairwise = coo_matrix((p_ij, (i, j)), shape=(s+1, s+1)).todense()

    '''Define the matrix of indipendent probabilities'''
    marginal = pairwise.sum(axis=1) #define the normalization using HiC data
    marginal = marginal / marginal.sum()
    indip = np.outer(marginal,marginal)

    info = np.log2(pairwise / indip)
    info = convertNansToZeros(coo_matrix(info)).todense()
    info = convertInfsToZeros(coo_matrix(info)).todense()

    counts = np.array(pairwise) * np.array(info)  #the mutual information matrix

    '''Filter with respect to centrality'''
    counts_1 = np.zeros(counts.shape)
    counts_2 = np.zeros(counts.shape)
    counts_3 = np.zeros(counts.shape)
    counts_4 = np.zeros(counts.shape)
    for row in xrange(counts.shape[0]):
        for col in xrange(counts.shape[1]):
                if (row in tags_1[:,0]) and (col in tags_1[:,0]):
                    counts_1[row,col] = counts[row,col]
                if (row in tags_2[:,0]) and (col in tags_2[:,0]):
                    counts_2[row,col] = counts[row,col]
                if (row in tags_3[:,0]) and (col in tags_3[:,0]):
                    counts_3[row,col] = counts[row,col]
                if (row in tags_4[:,0]) and (col in tags_4[:,0]):
                    counts_4[row,col] = counts[row,col]
    locals()['mi_chr'+chrom] = [np.sum(counts_1),np.sum(counts_2),np.sum(counts_3),np.sum(counts_4)]
    fig, ax = plt.subplots()
    y = locals()['mi_chr'+chrom]
    ax.scatter(x=range(len(y)),y=y,color='blue',label='HiC')
    ax.set_xticks([0,1,2,3])
    ax.set_xticklabels(["periphery","II layer","I layer","center"])
    ax.set_title('Mutual information for chr'+chrom)
    plt.legend()
    plt.savefig('MI_HiC_chr'+chrom+'.png')

# fig, ax = plt.subplots()
# y = np.array([mi_10min,mi_15min,mi_30min,mi_on])[:,3]
# ax.scatter(x=range(4), y=y,color='blue')
# ax.set_xticks([0,1,2,3])
# ax.set_xticklabels(["10min","15min","30min","ON"])
# ax.set_title('Mutual information for chr1 at the center')
# plt.savefig('MI_center_chr1.png')